In [1]:
import lxml.html as lh
import time
import urllib.request
import argparse
import datetime
import pytz
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException
import os
import coloredlogs
import pandas as pd
import numpy as np
import pandas.util.testing as tm
import altair as alt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  del sys.path[0]


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
os.listdir()

['.config', 'drive', 'sample_data']

In [0]:
os.chdir('drive/My Drive/intern_trading/trading_RL_yahoo')

In [5]:
from docopt import docopt
from trading_bot.ops import get_state
from trading_bot.agent import Agent
from trading_bot.methods import evaluate_model
from trading_bot.utils import (
    get_stock_data,
    format_currency,
    format_position,
    show_eval_result,
    switch_k_backend_device
)


Using TensorFlow backend.


In [0]:
tz = pytz.timezone('Asia/Kolkata')

path1 = os.getcwd()
path = path1 + '/chromedriver'

In [0]:
ignored_exceptions=(StaleElementReferenceException,)
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--headless')
options.add_experimental_option('excludeSwitches', ['enable-logging'])
#options.add_argument(f'user-agent={userAgent}')
#driver = webdriver.Chrome(executable_path=path , options=options)




In [0]:
 model_name = 'model_debug_50'
 test_stock = 'test_stock_rl.csv'
 total_stock='gold_stock_rl.csv'
 window_size = 10
 debug = True

In [9]:
# read csv into dataframe
df = pd.read_csv(test_stock)
# filter out the desired features
df = df[['Date', 'Adj Close']]
# rename feature column names
df = df.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df['date'] = dates

df.head()

,date,actual
0,2020-05-18,30028.98047
1,2020-05-19,30196.16992
2,2020-05-20,30818.60938
3,2020-05-21,30915.83984


In [10]:
# read csv into dataframe
df1 = pd.read_csv(total_stock)
# filter out the desired features
df1 = df1[['Date', 'Adj Close']]
# rename feature column names
df1 = df1.rename(columns={'Adj Close': 'actual', 'Date': 'date'})
# convert dates from object to DateTime type
dates = df1['date']
dates = pd.to_datetime(dates, infer_datetime_format=True)
df1['date'] = dates

df1.head()


,date,actual
0,2020-05-17,1767.900024
1,2020-05-18,1736.500000
2,2020-05-19,1749.000000
3,2020-05-20,1749.699951
4,2020-05-21,1733.699951


In [0]:
def Real(url,count):
    if count == 0:
        
        driver.get(url)
        #print(driver)

    else:
        driver.refresh()
        time.sleep(15)

    infile = driver.page_source
    doc = lh.fromstring(infile)
    live = doc.xpath('/html/body/div[1]/div/div/div[1]/div/div[2]/div/div/div[4]/div/div/div/div[3]/div/div/span[1]')
    live = float(live[0].text.replace(',',''))
    return live 

In [0]:
def main(args):
    count = 0
    total_profit = 0
    t=0
    history = []
    reward = 0
    ticker = args.ticker + '.NS'
    price = []
    window_size =10
    time_now = datetime.datetime.now(tz).time()
    while(datetime.time(0,0, tzinfo=tz) < time_now < datetime.time(23,59, tzinfo=tz)):
        test_stock = 'https://in.finance.yahoo.com/world-indices'.format(ticker,ticker)
        print(count)
        live = Real(test_stock,count)
        count+=1        
        price.append(live)
        if count < window_size:
           continue
        model_name='model_debug_50'  
        print(live)
        initial_offset = price[1] - price[0]
        state = get_state(price, 0, window_size + 1)
        next_state = get_state(price, t + 1, window_size + 1)
        agent = Agent(state_size=window_size, pretrained=True, model_name=model_name)
        agent.inventory = []
        

        profit = evaluate_model(agent,state,next_state, price, window_size=window_size)
        show_eval_result(model_name, profit, initial_offset)
        t+=1
        state = next_state

In [0]:
def evaluate_model(agent, state, next_state, data, t, total_profit, history, reward, window_size, debug=False):
  
    print(t)
        # select an action
    action = agent.act(state, is_eval=True)

        # BUY
    if action == 1:
        agent.inventory.append(data[t])

        history.append((data[t], "BUY"))
        if debug:
            logging.debug("Buy at: {}".format(format_currency(data[t])))

        # SELL
    elif action == 2 and len(agent.inventory) > 0:
        bought_price = agent.inventory.pop(0)
        delta = data[t] - bought_price
        reward = delta #max(delta, 0)
        total_profit += delta

        history.append((data[t], "SELL"))
        if debug:
            logging.debug("Sell at: {} | Position: {}".format(
                    format_currency(data[t]), format_position(data[t] - bought_price)))
        # HOLD
    else:
        history.append((data[t], "HOLD"))

#        done = (t == data_length - 1)
    agent.memory.append((state, action, reward, next_state))

    return total_profit


   




In [15]:
 import logging
 window_size = 10
 agent = Agent(window_size, pretrained=True, model_name=model_name)
 test_data = get_stock_data(test_stock)
 initial_offset = test_data[1] - test_data[0]
 
 test_result, history = evaluate_model(agent, test_data, window_size,debug=True)
 show_eval_result(model_name, test_result, initial_offset)

[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


In [0]:
def interpretation(df, history, title="stock daily session"):
    # add history to dataframe
    position = [history[0][0]] + [x[0] for x in history]
    actions = ['HOLD'] + [x[1] for x in history]
    df['position'] = position
    df['action'] = actions
    
    # specify y-axis scale for stock prices
    scale = alt.Scale(domain=(min(min(df['actual']), min(df['position'])) - 150, max(max(df['actual']), max(df['position'])) + 150), clamp=True)
    
    # plot a line chart for stock positions
    actual = alt.Chart(df).mark_line(
        color='green',
        opacity=0.5
    ).encode(
        x='date:T',
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale)
    ).interactive(
        bind_y=False
    )
    
    # plot the BUY and SELL actions as points
    points = alt.Chart(df).transform_filter(
        alt.datum.action != 'HOLD'
    ).mark_point(
        filled=True
    ).encode(
        x=alt.X('date:T', axis=alt.Axis(title='Date')),
        y=alt.Y('position', axis=alt.Axis(format='$.2f', title='Price'), scale=scale),
        color='action'
    ).interactive(bind_y=False)

    # merge the two charts
    graph = alt.layer(actual, points, title=title).properties(height=300, width=1000)
    
    return graph

In [17]:
graph = interpretation(df, history, title=test_stock)
graph

alt.LayerChart(...)

In [18]:
 coloredlogs.install(level='DEBUG')
 switch_k_backend_device()
 test_data = get_stock_data(total_stock)
 initial_offset = test_data[1] - test_data[0]

 test_result, history = evaluate_model(agent, test_data, window_size, debug)
 show_eval_result(model_name, test_result, initial_offset)

2020-05-21 10:31:58 6e5af76c70c3 root[2033] DEBUG switching to TensorFlow for CPU


[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]


2020-05-21 10:32:08 6e5af76c70c3 root[2033] DEBUG Buy at: $1736.50
2020-05-21 10:32:08 6e5af76c70c3 root[2033] DEBUG Sell at: $1749.00 | Position: +$12.50
2020-05-21 10:32:08 6e5af76c70c3 root[2033] INFO model_debug_50: +$12.50



In [19]:
graph = interpretation(df1, history, title=total_stock)
graph

alt.LayerChart(...)